## CNN trained on a custom dataset for image classification

In this notebook we are going to train an image classifier from scratch in PyTorch.

Before you proceed with this notebook, you need to collect the data for the classes you wish to create. To do so, you can follow any of the approaches presented in notebook ``04``.

Your data, should then exist in a folder with the path: 

``./data/my_image_dataset/``

Now you can proceed with the notebook. The following code has been adapted from notebook developed by [Terence Broad](https://github.com/terrybroad/CCI-AI-4-Media-23-24/blob/main/Week-3-CNNs-and-image-classification/train-image-classifier-from-scratch.ipynb).

In [ ]:
# If sklearn is not installed, execute the following command when in the correct conda env
#!conda install -c conda-forge scikit-learn

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.utils as vutils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#### Define our hyperparameters

In [ ]:
val_size = 0.3

batch_size = 100

momentum = 0.9

learn_rate = 0.001

num_epochs = 10

num_classes = 3 # change if you are working with a different number of classes

data_path = './data/my_image_dataset/'

##### Image transformations for the train dataset

Could you augment your training data by adding more transformations to them?

You could randomly change their brightness, contrast, saturation, and hue.

You could flip them horizontally or vertically with a 0.5 probability.

You could randomly rotate them.

Look in [here](https://pytorch.org/vision/stable/transforms.html) and [here](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py) for references and examples. 

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(size=(64, 64), antialias=True),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

##### Image transformations for the validation dataset

Do you need to also add the above transformations to your validation set? Or are the existing ones enough? You need to consider what the purpose of each dataset is.

In [ ]:
val_transform = transforms.Compose(
    [   
        torchvision.transforms.Resize(64, antialias=True),
        torchvision.transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

##### Create our train and test datasets

Here we create our train and validation datasets by splitting the full input dataset into two subsets, with a ratio that was defined with the val_size at the beginning of the notebook. A 70-30 split is quite common.

By setting a `random_state`, we are performing the split randomly but in a deterministic way, i.e. we will always get the same random train_test_split as long as we use the same random_state.

In [ ]:
# create train and validation datasets with seperate transforms
train_dataset = ImageFolder(data_path, transform=train_transform)
val_dataset = ImageFolder(data_path, transform=val_transform)

# get length of the full dataset before split, and save it in idx
num_train = len(train_dataset)

# create an array of idx numbers for each element of the full dataset
idx = list(range(num_train))
#print(num_train, idx)

# perform train / val split for data points
train_indices, val_indices = train_test_split(idx, test_size=val_size, random_state=42)

# override datasets to only be samples for each split
train_dataset = Subset(train_dataset, train_indices)
val_dataset = Subset(val_dataset, val_indices)

# create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

##### Plot sample images from the two datasets

In the next two cells we are plotting a sample of images from each dataset. See how the data augmentation transforms affect the images of the training set, compared to the images in the validation set.

In [ ]:
# get a batch of training images
sample_batch = next(iter(train_loader))[0][:64].to(device)

# create a grid of images
img_grid = vutils.make_grid(sample_batch, padding=2, normalize=True)

# convert to NumPy and transpose dimensions for matplotlib
img_grid_np = np.transpose(img_grid.cpu().numpy(), (1, 2, 0))

# plot the grid of images
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(img_grid_np)
plt.show()

In [ ]:
# get a batch of training images
sample_batch = next(iter(val_loader))[0][:64].to(device)

# create a grid of images
img_grid = vutils.make_grid(sample_batch, padding=2, normalize=True)

# convert to NumPy and transpose dimensions for matplotlib
img_grid_np = np.transpose(img_grid.cpu().numpy(), (1, 2, 0))

# plot the grid of images
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Validation Images")
plt.imshow(img_grid_np)
plt.show()

##### Define our CNN

In [ ]:
class ClassificationNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.conv3 = nn.Conv2d(128, 128, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

##### Setup core training objects

Look into available loss functions [here](https://pytorch.org/docs/stable/nn.html#loss-functions).

Look into available optimizers [here](https://pytorch.org/docs/stable/optim.html).

In [ ]:
model = ClassificationNetwork()
model.to(device)

# selecting cross entropy as the loss function for our classification
criterion = nn.CrossEntropyLoss()

# selecting stochastic gradient descent as our optimization algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate, momentum=momentum)

##### Training

What is different here from the training we did for the MNIST CNN classifier? 

In [ ]:
train_losses = []
val_losses = []

for epoch in range(num_epochs): 
    train_loss = 0.0
    
    # training loop
    for batch_idx, (data, target) in enumerate(train_loader):
        # get data
        inputs = data.to(device)
        labels = target.to(device)
        
        # zero the gradients
        optimizer.zero_grad()
        # forward pass
        predictions = model(inputs)
        # compute the loss
        loss = criterion(predictions, labels)
        # backpropagate
        loss.backward()
        # update the parameters, i.e. weights
        optimizer.step()

        # save statistics to plot later
        train_loss += loss.item()
    
    # validation loop
    with torch.no_grad():
        val_loss = 0.0
        for batch_idx, (data, target) in enumerate(val_loader):
            # get data
            inputs = data.to(device)
            labels = target.to(device)
            # forward pass, no backpropagation and optimisation
            predictions = model(inputs)
            # compute the loss
            loss = criterion(predictions, labels)
            # save statistics to plot later
            val_loss += loss.item()
    
    # normalise cumulative losses to dataset size
    train_loss = train_loss / len(train_loader)
    val_loss = val_loss / len(val_loader)
    
    # added cumulative losses to lists to plot later
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f'Epoch {epoch + 1}, train loss: {train_loss:.3f}, val loss: {val_loss:.3f}')

##### Plot training vs validation loss 

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Train vs validation loss")
plt.plot(train_losses,label="train")
plt.plot(val_losses,label="val")
plt.xlabel("epochs")
plt.ylabel("cumulative loss")
plt.legend()
plt.show()

##### Save the model

In [ ]:
torch.save(model.state_dict(), './models/img_classifier.pt')

### Tasks to do in-class and further explore at home

**Task 1:** Create your own dataset with 2, 3, or more classes, based on one of the suggested approaches. It would be effective to have at least 1000 images/class. Create a folder for each one of your classes and save the respective images there. Then move all of the class folders in `./data/my_image_dataset`. Make sure you manually clean-up your data before training, to remove any irrelevant or destroyed images.

**Task 2:** Run all the cells in this code to train a classifier on your custom dataset.

**Task 3:** Add image transformations on the training dataset. Look in [here](https://pytorch.org/vision/stable/transforms.html) and [here](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py) for references and examples. 

**Task 4:** Create a new notebook where you call the model that you just saved from this training and test it on some new unseen data. Look into the MNIST notebooks 01 and 02, as well as the PyTorch website, to figure out how you can call and use an already trained model.

**Bonus Challenges:**

**Bonus 1:** Look into the concept of Early Stopping. What is it? Could it be useful for our training? How? Attempt to implement it by adding the following lines of code after the training loop is completed:

   `if val_loss < best_loss:`
        
        `best_loss = val_loss`
        
        `torch.save(model.state_dict(), 'best_img_classifier.pt')`

For this to work, you will have to initialise best_loss with a high value before you enter the training loop.

**Bonus 2:** In this example you are building your classifier from scratch, i.e. you decide yourself what the architecture of the network is and you train it from the very beginning. Could you explore a way for training your classifier based on a pre-trained model? There are many available pre-trained models in [the torchvision models library](https://pytorch.org/vision/stable/models.html), like [ResNet](https://arxiv.org/abs/1512.03385) which is trained on [imagenet dataset](https://www.image-net.org/). This approach will require a few changes and additions in your notebook. Attempt it if you are feeling adventurous!

## Working in Google Colab

In order to be able to work in Google Colab with your notebooks and datasets, you need to mount your drive to the machine. Start the notebook with the commands below:

```python
from google.colab import drive
drive.mount('/content/drive')

# change directory using the os module
import os
os.chdir('drive/My Drive/')
os.listdir()                # shows the contents of the current dir
# os.mkdir("my-colab-repo") # creating a directory
# os.chdir("my-colab-repo") # moving to this directory
# os.getcwd()               # printing the current directory